In [1]:
import pandas as pd 
import numpy as np 
import statsmodels.tsa.stattools as ts
from statsmodels.tsa.vector_ar.vecm import coint_johansen
import matplotlib.pyplot as plt
from backtest_utiles import *
from itertools import combinations
from statsmodels.tsa.stattools import coint

/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [2]:
df = pd.read_csv('SPX_components_close_data.csv').set_index('Date')
df.index = pd.to_datetime(df.index)
df_last_yr = df[df.index>='2022-01-01']
coint_test_list = select_stock_pairs_from_correlation(df_period=df_last_yr, 
                                                        top_corr_pairs=15,
                                                        drop_correlation=0.97)
coint_pairs = find_cointegrated_pairs(df_period=df_last_yr, 
                                      coint_test_list=coint_test_list,
                                      p_value_threshold=0.01)

In [3]:
coint_pairs

[('PHM', 'LEN', 0.008034695794772112),
 ('MLM', 'LRCX', 0.005662451950343634),
 ('MLM', 'MAR', 0.0012817968759439994),
 ('LRCX', 'MAR', 0.00040579964354091786),
 ('MAR', 'VMC', 0.004909521644361649)]